In [173]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize
from torch.utils.data import Subset

In [174]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('./data', train=False, transform=transform)

In [175]:
# Splitting datasets for two tasks
train_dataset_task1 = Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] <= 5])
train_dataset_task2 = Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] > 5])
test_dataset_task1 = Subset(test_dataset, [i for i in range(len(test_dataset)) if test_dataset.targets[i] < 5])
test_dataset_task2 = Subset(test_dataset, [i for i in range(len(test_dataset)) if test_dataset.targets[i] >= 5])

In [176]:
unique_labels_1 = set()
unique_labels_2 = set()

for _, target in train_dataset_task1:
    unique_labels_1.add(target)

for _, target in train_dataset_task2:
    unique_labels_2.add(target)

print(f"First: {unique_labels_1}")
print(f"Second: {unique_labels_2}")

First: {0, 1, 2, 3, 4, 5}
Second: {8, 9, 6, 7}


In [177]:
train_dataloader_task1 = DataLoader(train_dataset_task1, batch_size=64, shuffle=True)
test_dataloader_task1 = DataLoader(test_dataset_task1, batch_size=256, shuffle=False)

for X, y in train_dataloader_task1:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [178]:
train_dataloader_task2 = DataLoader(train_dataset_task2, batch_size=64, shuffle=True)
test_dataloader_task2 = DataLoader(test_dataset_task2, batch_size=256, shuffle=False)

In [179]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [180]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, nn.Linear):
    #             nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
    #         elif isinstance(m, nn.Conv2d):
    #             nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [181]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [182]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")

In [183]:
model = NeuralNetwork(num_classes=10, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [184]:
epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_task1, model, loss_fn, optimizer)
  test(test_dataloader_task1, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model_old.pth")

Epoch 1
---------------------------
Loss: 2.318275,    64/36000
Loss: 0.276660,  6464/36000
Loss: 0.346340, 12864/36000
Loss: 0.211132, 19264/36000
Loss: 0.343147, 25664/36000
Loss: 0.272822, 32064/36000
Test Error: 
 Accuracy: 87.3, Avg Loss: 0.343310

Epoch 2
---------------------------
Loss: 0.367829,    64/36000
Loss: 0.167150,  6464/36000
Loss: 0.339057, 12864/36000
Loss: 0.272547, 19264/36000
Loss: 0.201382, 25664/36000
Loss: 0.185659, 32064/36000
Test Error: 
 Accuracy: 87.1, Avg Loss: 0.342122

Epoch 3
---------------------------
Loss: 0.228486,    64/36000
Loss: 0.124392,  6464/36000
Loss: 0.233071, 12864/36000
Loss: 0.249532, 19264/36000
Loss: 0.357625, 25664/36000
Loss: 0.437023, 32064/36000
Test Error: 
 Accuracy: 88.6, Avg Loss: 0.306197

Epoch 4
---------------------------
Loss: 0.243054,    64/36000
Loss: 0.124935,  6464/36000
Loss: 0.124939, 12864/36000
Loss: 0.110449, 19264/36000
Loss: 0.140190, 25664/36000
Loss: 0.248078, 32064/36000
Test Error: 
 Accuracy: 88.2, Avg 

In [185]:
def val(epoch):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader_task1):
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [186]:
val(1)

Validation Acc: 89.82



________

In [187]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model_old.pth"))

In [188]:
from copy import deepcopy

def get_fisher_diag(model, dataloader, params, empirical=False):
    fisher = {}
    params_dict = dict(params)
    for n, p in deepcopy(params_dict).items():
        p.data.zero_()
        fisher[n] = p.data.clone().detach().requires_grad_()

    model.eval()

    for input, gt_label in dataloader:
        input, gt_label = input.to(device), gt_label.to(device)
        model.zero_grad()
        output = model(input)

        if empirical:
            label = output.max(1)[1]
        else:
            label = gt_label

        # label = gt_label.repeat(output.size(0))
        negloglikelihood = torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(output, dim=1), label)
        negloglikelihood.backward()

        for n, p in model.named_parameters():
            fisher[n].data += p.grad.data ** 2 / len(dataloader.dataset)

    fisher = {n: p for n, p in fisher.items()}
    return fisher


def get_ewc_loss(model, fisher, p_old):
    loss = 0
    for n, p in model.named_parameters():
        _loss = fisher[n] * (p - p_old[n]) ** 2
        loss += _loss.sum()
    return loss

In [189]:
model.to(device)

ewc_lambda = 500_000

fisher_matrix = get_fisher_diag(model, train_dataloader_task1, model.named_parameters())
prev_params = {n: p.data.clone() for n, p in model.named_parameters()}

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

In [190]:
print(fisher_matrix)

{'fc1.weight': tensor([[1.6301e-08, 1.6300e-08, 1.6294e-08,  ..., 1.6124e-08, 1.6255e-08,
         1.6296e-08],
        [6.2606e-07, 6.2606e-07, 6.2592e-07,  ..., 5.8690e-07, 6.1910e-07,
         6.2564e-07],
        [1.4374e-07, 1.4373e-07, 1.4374e-07,  ..., 1.4306e-07, 1.4363e-07,
         1.4371e-07],
        ...,
        [3.6598e-07, 3.6601e-07, 3.6603e-07,  ..., 3.5811e-07, 3.6451e-07,
         3.6574e-07],
        [9.1493e-08, 9.1490e-08, 9.1445e-08,  ..., 8.8371e-08, 9.1086e-08,
         9.1487e-08],
        [3.2441e-07, 3.2441e-07, 3.2436e-07,  ..., 2.9740e-07, 3.1868e-07,
         3.2434e-07]], requires_grad=True), 'fc1.bias': tensor([1.6302e-08, 6.2606e-07, 1.4373e-07, 2.4174e-08, 1.9960e-10, 3.0725e-07,
        1.1680e-07, 4.1918e-08, 1.0328e-07, 1.5689e-08, 1.6209e-07, 9.6254e-09,
        5.4251e-09, 2.2509e-07, 7.0144e-10, 1.1952e-08, 1.5106e-07, 1.6801e-07,
        2.1807e-08, 1.8777e-08, 2.1331e-08, 2.3167e-07, 5.8824e-08, 2.0246e-08,
        3.3673e-09, 1.1142e-08, 7.78

In [191]:
def train(dataloader, model, loss_fn, optimizer, fisher_matrix, prev_params):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)

        # Original loss
        ce_loss = loss_fn(pred, y)

        # EWC loss
        ewc_loss = get_ewc_loss(model, fisher_matrix, prev_params)

        loss = ce_loss + ewc_lambda * ewc_loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [192]:
def val(epoch):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader_task1):
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted_old = outputs.max(1)
            print(torch.unique(predicted_old))
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [193]:
for epoch in range(10):
    print(f"Epoch {epoch+1}: ----------------------")
    train(train_dataloader_task2, model, loss_fn, optimizer, fisher_matrix, prev_params)
    test(test_dataloader_task2, model, loss_fn)
print("Done!")

Epoch 1: ----------------------
Loss: 13.574834,    64/24000
Loss: 3.256116,  6464/24000
Loss: 2.654275, 12864/24000
Loss: 2.459614, 19264/24000
Test Error: 
 Accuracy: 32.8, Avg Loss: 2.174725

Epoch 2: ----------------------
Loss: 2.337003,    64/24000
Loss: 2.032220,  6464/24000
Loss: 1.761280, 12864/24000
Loss: 1.554754, 19264/24000
Test Error: 
 Accuracy: 44.5, Avg Loss: 1.721210

Epoch 3: ----------------------
Loss: 1.518960,    64/24000
Loss: 1.375294,  6464/24000
Loss: 1.237617, 12864/24000
Loss: 1.081668, 19264/24000
Test Error: 
 Accuracy: 65.1, Avg Loss: 1.673280

Done!


In [194]:
val(1)

tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 6, 7, 8])
tensor([0, 1, 2, 3, 4, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 4, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
tensor([0, 1, 2, 3, 6, 7, 8])
Validation Acc: 27.54

